### Yoly Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from cv2.ximgproc import guidedFilter
from skimage.color import rgb2hsv
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import sys

from utils.collection import HazeCollection
from utils.network import VaeNet, Net
from utils.loss import StdLoss
from utils.utils import get_atmosphere, torch_to_np, np_to_torch, save_image, get_information
from utils.imresize import np_imresize

#### Collection

In [2]:
haze_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/synthetic"
image_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/original"

haze_path_dt2 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/real-world"

haze_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/hazy_val"
image_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/gt_val"

In [3]:
size = 256
channels = 3

collection_dt1 = HazeCollection(haze_path_dt1, image_path_dt1)
collection_dt2 = HazeCollection(haze_path_dt2)
collection_dt3 = HazeCollection(haze_path_dt3, image_path_dt3)

#### Model

In [4]:
# UNet
un = Net(channels)

x = torch.zeros([1, channels, size, size])
a = un(x)
a.shape

torch.Size([1, 3, 256, 256])

#### Training

In [5]:
class YolyTrainer:
    def __init__(
        self,
        metrics=[]
    ):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.metrics = metrics

    def init(self, image):
        self.model = {
            "netJ": Net(out_channel=3).to(self.device),
            "netT": Net(out_channel=1).to(self.device),
            "netA": VaeNet(image.shape).to(self.device)
        }
        self.loss = {
            "mse": nn.MSELoss().to(self.device),
            "std": StdLoss().to(self.device)
        }

        atmosphere = get_atmosphere(image)
        self.valA = nn.Parameter(data=torch.cuda.FloatTensor(
            atmosphere.reshape((1, 3, 1, 1))), requires_grad=False)

        parameters = [p for p in self.model["netJ"].parameters()]
        parameters += [p for p in self.model["netT"].parameters()]
        parameters += [p for p in self.model["netA"].parameters()]

        self.optimizer = optim.Adam(
            parameters,
            lr=self.learning_rate
        )

    def t_matting(self, mask_out_np, original):
        refine_t = guidedFilter(original.transpose(1, 2, 0).astype(
            np.float32), mask_out_np[0].astype(np.float32), 50, 1e-4)
        return np.array([np.clip(refine_t, 0.1, 1)])

    def train(self, information, epochs):
        (haze, image, name) = information
        self.init(haze)
        loop = tqdm(range(epochs))

        losses = []
        lossFinal = None

        hazeTorch = np_to_torch(haze)
        hazeTorch = hazeTorch.to(device=self.device)
        for i in loop:
            # forward
            predT = self.model["netT"](hazeTorch)
            predJ = self.model["netJ"](hazeTorch)
            predA = self.model["netA"](hazeTorch)

            #lossT = self.loss["std"](predT)
            lossJ = self.loss["mse"](predT * predJ + (1 - predT) * predA, hazeTorch)
            lossA = self.model["netA"].getLoss()

            hsv = np_to_torch(rgb2hsv(torch_to_np(predJ).transpose(1, 2, 0)))
            cap_prior = hsv[:, :, :, 2] - hsv[:, :, :, 1]
            cap_loss = self.loss["mse"](cap_prior, torch.zeros_like(cap_prior))

            lossFinal = lossJ + lossA
            lossFinal += 1.0 * cap_loss
            lossFinal += 0.1 * self.loss["std"](predA)
            lossFinal += self.loss["mse"](predA,self.valA * torch.ones_like(predA))

            #if i % 100 == 0 or i == epochs-1:
            if i == epochs-1:
                imageJ = np.clip(torch_to_np(predJ), 0, 1)
                imageT = np.clip(torch_to_np(predT), 0, 1)
                imageA = np.clip(torch_to_np(predA), 0, 1)
                imageT = self.t_matting(imageT, haze)

                # I(x) = J(x)t(x) + A(1 − t(x))
                post = np.clip((haze - ((1 - imageT) * imageA)) / imageT, 0, 1)
                post = np_imresize(post, output_shape=haze.shape[1:])
                save_image('{}_{}'.format(name[0].split('.')[0], i+1), post, './images/yoly/' + self.dt_number)

            # backward
            self.optimizer.zero_grad()
            lossFinal.backward(retain_graph=True)
            self.optimizer.step()

            # update tqdm
            loop.set_postfix(loss=lossFinal.item())
            losses.append(lossFinal.item())
        
        # metrics
        finalSSIM, finalPSNR = 0, 0
        if image is not None:
            finalSSIM = ssim(post, image, channel_axis=0, data_range=1)
            finalPSNR = psnr(post, image)
            print("Ssim: {:.4f}, Psnr: {:.4f}\n".format(finalSSIM, finalPSNR))

        return [np.sum(losses)/len(losses), finalSSIM, finalPSNR]

    def fit(
        self,
        collection,
        epochs=1,
        dt_number='',
        learning_rate=1e-3
    ):
        historial = []
        self.dt_number = dt_number
        self.learning_rate = learning_rate
        #loop = enumerate(tqdm(collection))
        for i, info in enumerate(collection):
            print('Image {}/{} [{}]'.format(i+1, len(collection), info[2][0]))
            loss, ssim, psnr = self.train(info, epochs)
            historial.append([loss, ssim, psnr])
        return historial

In [6]:
dehazing = YolyTrainer()

#### Testing

In [7]:
epochs = 512
learning_rate = 1e-4

##### D1

In [8]:
historial_dt1 = dehazing.fit(collection_dt1, epochs, 'd1/', learning_rate)
get_information(historial_dt1, collection_dt1)

Image 1/10 [1381.jpg]


100%|██████████| 512/512 [00:25<00:00, 20.39it/s, loss=0.0901]


Ssim: 0.8456, Psnr: 18.1391

Image 2/10 [5576.jpg]


100%|██████████| 512/512 [00:18<00:00, 28.18it/s, loss=0.401]


Ssim: 0.8039, Psnr: 20.8515

Image 3/10 [7471.jpg]


...

Ssim: 0.7684, Psnr: 19.3270

Image 9/10 [1352.jpg]


100%|██████████| 512/512 [00:47<00:00, 10.80it/s, loss=0.24] 


Ssim: 0.3921, Psnr: 12.6644

Image 10/10 [4561.jpg]


100%|██████████| 512/512 [00:19<00:00, 26.25it/s, loss=0.132]

Ssim: 0.9444, Psnr: 27.9635

Mean SSIM: 0.7523489892482758
Mean PSNR: 19.708888375267897
Best SSIM: 4561.jpg
Best PSNR: 4561.jpg


##### D2

In [9]:
historial_dt2 = dehazing.fit(collection_dt2, epochs, 'd2/', learning_rate)
get_information(historial_dt2, collection_dt2)

Image 1/10 [SFC_Google_197.jpeg]


100%|██████████| 512/512 [00:36<00:00, 13.96it/s, loss=0.209]


Image 2/10 [MLS_Bing_117.jpeg]


100%|██████████| 512/512 [00:40<00:00, 12.51it/s, loss=0.468]


Image 3/10 [HazyDr_Google_396.jpeg]


...

Image 9/10 [KRO_Google_143.jpeg]


100%|██████████| 512/512 [00:40<00:00, 12.54it/s, loss=0.37] 


Image 10/10 [HazeDr_Google_404.jpeg]


100%|██████████| 512/512 [01:42<00:00,  5.00it/s, loss=0.255]

Mean SSIM: 0.0
Mean PSNR: 0.0
Best SSIM: SFC_Google_197.jpeg
Best PSNR: SFC_Google_197.jpeg


##### D3

In [10]:
historial_dt3 = dehazing.fit(collection_dt3, epochs, 'd3/', learning_rate)
get_information(historial_dt3, collection_dt3)

Image 1/10 [1410_5.png]


  6%|▌         | 30/512 [00:01<00:28, 16.74it/s, loss=0.317]

100%|██████████| 512/512 [00:30<00:00, 16.64it/s, loss=0.147]


Ssim: 0.8990, Psnr: 20.5653

Image 2/10 [1430_5.png]


100%|██████████| 512/512 [00:30<00:00, 16.61it/s, loss=0.0755]


...

Ssim: 0.8567, Psnr: 21.0872

Image 9/10 [1425_5.png]


100%|██████████| 512/512 [00:30<00:00, 16.53it/s, loss=0.142]


Ssim: 0.7359, Psnr: 17.4262

Image 10/10 [1420_5.png]


100%|██████████| 512/512 [00:31<00:00, 16.46it/s, loss=0.354]


Ssim: 0.7677, Psnr: 12.7352

Mean SSIM: 0.8044904470443726
Mean PSNR: 18.479269258750783
Best SSIM: 1415_5.png
Best PSNR: 1415_5.png
